Importing Libararies

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

C:\Users\Mantas\AppData\Local\Temp\ipykernel_15804\2841259704.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Load data and seperate target

In [2]:
file_path = './train.csv'
data = pd.read_csv(file_path)

variable init

In [3]:
features = ['LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = data[features]
y = data.SalePrice

trainX, validateX, trainY, validateY = train_test_split(X, y, random_state=0)

Test model

In [4]:
model = RandomForestRegressor(random_state=0)
model.fit(trainX, trainY)

predictions = model.predict(validateX)
mae = mean_absolute_error(validateY, predictions)

print(mae)

20185.36428219178


Test on full data set

In [5]:
test_data = pd.read_csv('./test.csv')
final_predictions = model.predict(test_data[features])

Generate Kaggle Housing Prices Competition Submission

In [6]:
output = pd.DataFrame({'Id': test_data.Id, 'SalePrice': final_predictions})
output.to_csv('submission.csv', index=False)